# (Unofficial) Baseball Stats API

## Goal: Excel spreadsheet that has last 10 sesasons stats for all players
- If you want to only look at last years stats, you can filter by '2021'
- If you want to look at a players career stats, you can filter by player/player_id

In [ ]:
from baseball import Baseball


player_id

In [ ]:
player_data

In [ ]:
player_details = Baseball().player_data.get_player_details("493316")
player_details

In [ ]:
search_results = stats.search_player_all("Trea Turner")

player_id = search_results[0].get("player_id")
player_info = stats.player_info(player_id)

player_info

In [ ]:
import requests
from urllib.parse import urlencode, quote_plus
from baseball import Baseball


player_data = Baseball().player_data.search_for_players("Aaron Judge")
player_id = player_data[0]["player_id"]

params = {}
params["player_id"] = player_id
params["game_type"] = f"'R'"
params["season"] = "2015"
params["league_list_id"] = "'mlb'"

encoded_params = urlencode(params, quote_via=quote_plus)

url = f"http://lookup-service-prod.mlb.com/json/named.sport_hitting_tm.bam?{encoded_params}"
r = requests.get(url)
r.json()["sport_hitting_tm"]["queryResults"].get("row")

### Get year by year stats by player name

In [ ]:
from baseball import Baseball
import pandas as pd


player_data = Baseball().player_data.search_for_players("Christian Yelich")
player_id = player_data[0]["player_id"]


seasons = ["2021", "2020", "2019", "2018", "2017", "2016"]

player_stats = []
for year in seasons:
    hitting_stats = Baseball().stats_data.season_hitting_stats(player_id, season=year)
    if hitting_stats is None:
        break
    else:
        player_stats.append(hitting_stats)

df = pd.DataFrame(player_stats)
df


### Get Teams by Season

In [ ]:
from baseball import api


season_teams = api.TeamData().get_teams_by_season("2022")
season_teams


In [ ]:
api.TeamData().get_teams_by_season("2022")

In [ ]:
import pandas as pd


df = pd.DataFrame(season_teams)
df

### Get 40 man roster

In [ ]:
import pandas as pd


df = pd.DataFrame(mlb_roster)
df.to_excel("mlb_roster.xlsx")

In [ ]:
from baseball import api
from tqdm import tqdm, trange
from datetime import datetime, date
import time


def calculate_age(birth_date, season_year):
    today = date(year=int(season_year), month=4, day=1)
    age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    return age



team_list = api.TeamData().get_teams_by_season("2022")

mlb_roster = []
for team in trange(len(team_list)):
    team_id = team_list[team].get("team_id")
    team_roster = api.TeamData().get_40_man_roster(team_id)
    mlb_roster.extend(team_roster)
    time.sleep(.1)

print(f"Found {len(mlb_roster)} total players.")

seasons = ["2022", "2021", "2020", "2019", "2018", "2017", "2016"]

all_stats = []
for player in trange(len(mlb_roster)):
    player_position = mlb_roster[player]["position_txt"]
    if player_position == "P":
        time.sleep(.25)
        continue
    else:
        player_id = mlb_roster[player]["player_id"]
        player_name = mlb_roster[player]["name_display_first_last"]
        player_debut = mlb_roster[player]["pro_debut_date"][0:4]

        birth_date_str = mlb_roster[player]["birth_date"][0:10]
        birth_date_dt = datetime.strptime(birth_date_str, "%Y-%m-%d")
        for year in seasons:
            if year == "2022":
                year_stats = {
                    'team_id': mlb_roster[player]["team_id"],
                    'season': '2022',
                    'sport_code': 'mlb',
                    'team_full': mlb_roster[player]["team_name"],
                    'team_abbrev': mlb_roster[player]["team_abbrev"],
                    'player_id': mlb_roster[player]["player_id"],
                    'sport': 'MLB',
                    'player_name': mlb_roster[player]["name_display_first_last"],
                    'player_age': calculate_age(birth_date_dt, year)
                }
                all_stats.append(year_stats)
                continue

            if player_debut == "" or int(year) < int(player_debut):
                break
            else:
                year_stats = api.StatsData().season_hitting_stats(player_id, year)
                if year_stats is None:
                    continue
                if type(year_stats) == list:
                    for i in year_stats:
                        i["player_age"] = calculate_age(birth_date_dt, year)
                        i["player_name"] = player_name
                    all_stats.extend(year_stats)
                else:
                    year_stats["player_age"] = calculate_age(birth_date_dt, year)
                    year_stats["player_name"] = player_name
                    all_stats.append(year_stats)
    time.sleep(.25)

len(all_stats)

In [ ]:
import pandas as pd


df = pd.DataFrame(mlb_roster)
df.to_excel("mlb_roster.xlsx")
